In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

from random import sample
import random

import time


In [2]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [3]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [4]:
def gera_particoes(N, tam_part, tam_jan):
	
	temp = tam_part-tam_jan
	subset = []
	 
	for i in range(0, N, temp):
		if i+tam_part > N:
			subset.append([k for k in range(i, N)])
		else:
			subset.append([k for k in range(i, i+tam_part)])
	
	return subset

In [5]:
MAX_CPU_TIME = 3600.0
EPSILON = 1.e-6

def relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, parts, yp_sol, yr_sol):

	try:

		# create a new model
		model = gp.Model("clspr")

		# create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		for i in range(N):
			if i < min(parts):
				# fixadas
				yp[i].lb = yp_sol[i]
				yp[i].ub = yp_sol[i]
				yr[i].lb = yr_sol[i]
				yr[i].ub = yr_sol[i]
			elif i in parts:
				# binary
				yp[i].VType = gp.GRB.BINARY
				yr[i].VType = gp.GRB.BINARY
		
		#print("Press any key to continue...")
		#input()	
		
		model.update()

		# set objective

		#model.setObjective(
		#	gp.quicksum(PP[i]*xp[i] + HP[i]*sp[i] + FP[i]*yp[i] + PR[i]*xr[i] + HR[i]*sr[i] + FR[i]*yr[i] for i in range(N)), 
		#	sense=GRB.MINIMIZE
		#	)
		
		fobj = gp.quicksum(PP[i]*xp[i] for i in range(N))
		fobj += gp.quicksum(FP[i]*yp[i] for i in range(N))
		fobj += gp.quicksum(HP[i]*sp[i] for i in range(N))
		fobj += gp.quicksum(PR[i]*xr[i] for i in range(N))
		fobj += gp.quicksum(FR[i]*yr[i] for i in range(N))
		fobj += gp.quicksum(HR[i]*sr[i] for i in range(N))
	
		model.setObjective(fobj, sense = GRB.MINIMIZE)

		# add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(1,N))
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(1,N))
		
		model.addConstrs(xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i], SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
		
		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

    	# turn off display
		gp.setParam('OutputFlag', 0)

		# open log file
		_ = open('std_rf.log', 'w')
		
		# export .lp/.mps
		#model.write("clspr.lp")

		# optimize model
		model.optimize()

		objval = model.ObjVal

		yp_val = [yp[i].X for i in range(N)]
		yr_val = [yr[i].X for i in range(N)]

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return objval, yp_val, yr_val

In [6]:
def main(file_name, tam_part, tam_jan):

	solution = 0.0
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()
	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	start_time = time.time()

	particao = gera_particoes(N, tam_part, tam_jan)
	
	#print("numero de particoes:", len(particao))
	#for it in particao:
	#	print(it)

	yp_sol = [0]*N
	yr_sol = [0]*N
	
	for subset in particao:
		solution, yp_sol, yr_sol = relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, subset, yp_sol, yr_sol)
		 
	run_time = time.time() - start_time
	
	arquivo = open(os.path.join(result,'clsr_std_rf.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(solution,2))+';'
			   +str(round(run_time,2))+'\n')

	arquivo.close()

In [8]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,11):
			file_name = f"c{dim}_{id}.txt"
			tam_part = 5
			tam_jan = 3
			
			main(file_name, tam_part, tam_jan)